<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Packages" data-toc-modified-id="Packages-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Packages</a></span></li><li><span><a href="#Test-a-first-naive-bayesian-model" data-toc-modified-id="Test-a-first-naive-bayesian-model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Test a first naive bayesian model</a></span></li></ul></div>

## Packages

In [ ]:
%load_ext autoreload
%autoreload 2

## Test a first naive bayesian model

In [1]:
from models.naive_bayes_k import naive_bayes_k

In [3]:
import pandas as pd

In [ ]:
all_results = []
for taxonomy_level in range(7):
    for selected_primer in ['V1','V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V12', 'V23', 'V34', 'V45', 'V56', 'V67', 'V78', 'V89']:
        test_size, main_class_prop, accuracy = naive_bayes_k(k=5,sequence_origin='DairyDB', 
                                                           primers_origin='DairyDB', 
                                                           taxonomy_level=taxonomy_level,
                                                           selected_primer=selected_primer)
        all_results.append([selected_primer, taxonomy_level, test_size, main_class_prop, accuracy])
final_df = pd.DataFrame(all_results, columns=['HyperVariable Region','Taxonomy Rank to be classified','Test Size', 'Main Class prop', 'Naive Bayes - NB(4)'])
final_df.to_csv('results/models/Naive Bayes - NB(4) - DB - DB - accuracies.csv', index=False)

C:\Users\admin\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


------------------------------------ PARAMETER INFORMATION FOR NAIVE BAYES - NB(5) -------------------------------------
----------------------------------------------------- DATA ORIGINS -----------------------------------------------------
Sequence origin: DairyDB
Primers origin: DairyDB
------------------------------------ CHOSEN HYPERVARIABLE REGION AND TAXONOMY RANK -------------------------------------
HyperVariable Region: V1
Taxonomy Rank: 0
---------------------------------------------------- PREPROCESSING -----------------------------------------------------
Preprocessing description: Computing frequency of 5-mer (ATCG) in every sequence
--------------------------------------------------- MODEL PARAMETERS ---------------------------------------------------
Parameter dict: {'priors': None, 'var_smoothing': 1e-09}
Size of test set: 470
Part of test size compared to total: 0.2
------------------------------------------------------- RESULTS ---------------------------------------

C:\Users\admin\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


------------------------------------ PARAMETER INFORMATION FOR NAIVE BAYES - NB(5) -------------------------------------
----------------------------------------------------- DATA ORIGINS -----------------------------------------------------
Sequence origin: DairyDB
Primers origin: DairyDB
------------------------------------ CHOSEN HYPERVARIABLE REGION AND TAXONOMY RANK -------------------------------------
HyperVariable Region: V2
Taxonomy Rank: 0
---------------------------------------------------- PREPROCESSING -----------------------------------------------------
Preprocessing description: Computing frequency of 5-mer (ATCG) in every sequence
--------------------------------------------------- MODEL PARAMETERS ---------------------------------------------------
Parameter dict: {'priors': None, 'var_smoothing': 1e-09}
Size of test set: 1222
Part of test size compared to total: 0.2
------------------------------------------------------- RESULTS --------------------------------------

C:\Users\admin\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

C:\Users\admin\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


------------------------------------ PARAMETER INFORMATION FOR NAIVE BAYES - NB(5) -------------------------------------
----------------------------------------------------- DATA ORIGINS -----------------------------------------------------
Sequence origin: DairyDB
Primers origin: DairyDB
------------------------------------ CHOSEN HYPERVARIABLE REGION AND TAXONOMY RANK -------------------------------------
HyperVariable Region: V2
Taxonomy Rank: 6
---------------------------------------------------- PREPROCESSING -----------------------------------------------------
Preprocessing description: Computing frequency of 5-mer (ATCG) in every sequence
--------------------------------------------------- MODEL PARAMETERS ---------------------------------------------------
Parameter dict: {'priors': None, 'var_smoothing': 1e-09}
Size of test set: 1268
Part of test size compared to total: 0.2
------------------------------------------------------- RESULTS --------------------------------------

C:\Users\admin\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


------------------------------------ PARAMETER INFORMATION FOR NAIVE BAYES - NB(5) -------------------------------------
----------------------------------------------------- DATA ORIGINS -----------------------------------------------------
Sequence origin: DairyDB
Primers origin: DairyDB
------------------------------------ CHOSEN HYPERVARIABLE REGION AND TAXONOMY RANK -------------------------------------
HyperVariable Region: V3
Taxonomy Rank: 6
---------------------------------------------------- PREPROCESSING -----------------------------------------------------
Preprocessing description: Computing frequency of 5-mer (ATCG) in every sequence
--------------------------------------------------- MODEL PARAMETERS ---------------------------------------------------
Parameter dict: {'priors': None, 'var_smoothing': 1e-09}
Size of test set: 1684
Part of test size compared to total: 0.2
------------------------------------------------------- RESULTS --------------------------------------

C:\Users\admin\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
